In [1]:
# ! conda install -c conda-forge imbalanced-learn
# ! pip install nose
# ! pip install imbalanced-ensemble           

In [1]:
import re

def read_dot_dat_file(path):
    datContent = [i.strip().split() for i in open(path).readlines()]
    r = re.compile("@inputs.*")
    _at_data = datContent.index(['@data'])
    assert datContent[0][0] == '@relation'
    assert datContent[_at_data-1][0] == '@outputs'
    assert datContent[_at_data-2][0] == '@inputs'
    print(datContent[_at_data-3][2:])
    assert len(datContent[_at_data-3][2:]) == 2   # Two Class

    col_names = datContent[_at_data-2][1:]
    col_names.append(datContent[_at_data-1][1])
    
    df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=r', ', engine='python')
    # df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=", ", engine='python')

    class1 = datContent[_at_data-3][2:][0].replace("{","").replace(",","")
    class2 = datContent[_at_data-3][2:][1].replace("}","").replace(",","")

    df['Class'] = df['Class'].replace({class1: 1, class2: -1})
    
    return df



# temp

In [21]:
# import pandas as pd 
# import numpy as np
# import os

# base_path = 'Datasets/'
# need_to_convert = ['ecoli-0_vs_1-5-fold','haberman-5-fold','page-blocks0-5-fold','pima-5-fold','vehicle2-5-fold']

# for needed in need_to_convert:
#     for datFile in os.listdir(base_path+needed):
#         print(base_path+needed+'/'+datFile)
#         df = read_dot_dat_file(base_path+needed+'/'+datFile)
#         df.to_excel(base_path+needed+'/'+datFile.split('.')[0]+'.xlsx')


In [2]:
import time
import os
import glob
import numpy as np
import random  
import pandas as pd 
import random

from sklearn.tree import DecisionTreeClassifier 

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc,f1_score,matthews_corrcoef
from imblearn.metrics import geometric_mean_score


from imblearn.ensemble import RUSBoostClassifier
from imbalanced_ensemble.ensemble import SMOTEBoostClassifier
from maatpy.classifiers import AdaCost


In [7]:
base_path = 'Datasets/'
datasets  = os.listdir(base_path) 
classifiers = {"RUS": RUSBoostClassifier(random_state=0, algorithm='SAMME', estimator=DecisionTreeClassifier(max_depth=10)),
              "SMOTE": SMOTEBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10), n_estimators = 100, algorithm='SAMME', random_state=0),
              "Ada1": AdaCost(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=100, algorithm='adac1',random_state=0),
              "AdaCost": AdaCost(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=100, algorithm='adacost',random_state=0),}


def clasify(dataset, classifier):
    mcc = []
    f1 = []
    auc_a = []
    gmean = []
    times = []
    y_preds = []
    traFiles = sorted(glob.glob(base_path+dataset+'/*tra.xlsx'))
    tstFiles = sorted(glob.glob(base_path+dataset+'/*tst.xlsx'))
    for traPath, tstPath in zip(traFiles, tstFiles):
        # print(traPath)
        
        df_train = pd.read_excel(traPath)
        df_test = pd.read_excel(tstPath)

        x_train= df_train.iloc[:, 0:-1]
        y_train = df_train.iloc[:, -1]
        x_test= df_test.iloc[:, 0:-1]
        y_test = df_test.iloc[:, -1]
        
        #####
        # for some dataset get error Unknown label type: 'unknown'
        y_train = y_train.astype('int')
        y_test = y_test.astype('int')
        
        st = time.time()
        clf = classifiers[classifier]

        clf.fit(x_train, y_train)

        y_pred = clf.predict(x_test) 
        et = time.time()
        y_preds.append(y_pred)
        # compute error
        mcc.append(matthews_corrcoef(y_test, y_pred))
        #--------------------------------
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auc_a.append(auc(fpr, tpr))
        #--------------------------------
        f1.append(f1_score(y_test, y_pred))
        #--------------------------------
        gmean.append(geometric_mean_score(y_test, y_pred, labels=[1, -1]))
        
        #time of train and test
        times.append(et - st)
        
    return {"mcc": mcc, "auc": auc_a, "f1": f1, "gmean": gmean, "exe_time": times, "y_pred": y_pred}

aaaaaa =  clasify(datasets[0], 'Ada1')
aaaaaa

{'mcc': [-0.056313829183370835,
  -0.056313829183370835,
  0.19936606554975947,
  0.23095609134944634,
  0.7132259743375],
 'auc': [0.47674418604651164,
  0.47674418604651164,
  0.5717054263565892,
  0.6317829457364341,
  0.8274509803921568],
 'f1': [0.0, 0.0, 0.2222222222222222, 0.28571428571428575, 0.7272727272727272],
 'gmean': [0.0,
  0.0,
  0.4034732923929645,
  0.5568460463897045,
  0.8116794499134278],
 'exe_time': [0.030020475387573242,
  0.02517867088317871,
  0.016266822814941406,
  0.021300792694091797,
  0.02183365821838379],
 'y_pred': array([-1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1,  1,  1, -1,  1,  1])}

In [16]:
# #temp 
# for i in range(1,len(datasets)):
#     os.mkdir('./Results/'+datasets[i])
#     # print(os.path.isdir('./Results/'+datasets[i]))


In [4]:
def res_to_files(dataset, dict_res):
    np.save('./Results/'+dataset+'/mcc.npy',dict_res['mcc'])
    np.save('./Results/'+dataset+'/auc.npy',dict_res['auc'])
    np.save('./Results/'+dataset+'/gmean.npy',dict_res['gmean'])
    np.save('./Results/'+dataset+'/exe_time.npy',dict_res['exe_time'])
    np.save('./Results/'+dataset+'/y_pred.npy',dict_res['y_pred'])
    

In [8]:
for classifier in classifiers:
    for dataset in datasets:
        if not os.path.exists('./Results/'+dataset+'/y_pred.npy'):
            try:
                tmp_res = clasify(dataset, classifier)
                res_to_files(dataset, tmp_res)
            except:
                print(dataset)
            

led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold
ecoli-0-3-4-6_vs_5-5-fold
abalone19-5-fold
led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold
ecoli-0-3-4-6_vs_5-5-fold
abalone19-5-fold
led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold
ecoli-0-3-4-6_vs_5-5-fold
abalone19-5-fold
led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold
ecoli-0-3-4-6_vs_5-5-fold
abalone19-5-fold
